<a href="https://colab.research.google.com/github/davidraamirez/GradientWithoutBackpropagation/blob/main/LogisticRegression_fwd_gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
import tqdm
import torch.distributions as distr

In [2]:
%pip install torchmetrics --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 13.7 MB/s eta 0:00:00


In [3]:
import torchmetrics
import torchvision
from torchvision import transforms as T

Loading and preprocessing the data

In [4]:
#Load the dataset
train_data = torchvision.datasets.KMNIST('./data', train=True, download=True)

  0%|          | 0/18165135 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/train-images-idx3-ubyte.gz to ./data/KMNIST/raw



  0%|          | 0/29497 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/train-labels-idx1-ubyte.gz to ./data/KMNIST/raw



  0%|          | 0/3041136 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/KMNIST/raw



  0%|          | 0/5120 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/KMNIST/raw



In [5]:
# This loads data with both data conversion.
train_data = torchvision.datasets.KMNIST('./data', train=True, transform=T.ToTensor())

In [6]:
# Loaders are used to shuffle, batch, and possibly sample the elements of the dataset
train_loader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True)

In [7]:
xb, yb = next(iter(train_loader))
print(xb.shape)
print(yb.shape)

torch.Size([8, 1, 28, 28])
torch.Size([8])


In [8]:
# Loading the test data is similar, but (a) we do not apply data augmentation,
# and (b) we do not shuffle when building the mini-batches.
test_data = torchvision.datasets.KMNIST('./data', train=False, transform=T.ToTensor())
test_loader = torch.utils.data.DataLoader(test_data, batch_size=8, shuffle=False)

Define Logistic Regression

In [9]:
from torch import nn
from torch.nn import functional as F

In [10]:
class SimpleLogisticRegression(nn.Module):
  def __init__(self, input_size, w, b):
    super(SimpleLogisticRegression, self).__init__()
    input_size = 28
    self.linear = nn.Linear(input_size, 28*28)
    self.linear.weight = nn.Parameter(w)
    self.linear.bias = nn.Parameter(b)


  def forward(self, x):
    x = x.reshape(-1, 28*28)
    return torch.softmax(self.linear(x), 1)

In [11]:
# We check if CUDA is available. If you do not see it,
# activate a GPU from Runtime >> Change runtime type and 
# restart the notebook.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


Initialize the parameters

In [24]:
# We initialize the parameters randomly and the model with an input size
w = torch.FloatTensor(10, 28*28).uniform_(-1, 1)
#w = torch.randn((10, 28*28), requires_grad=False)
b = torch.FloatTensor(10).uniform_(-1, 1)
#b = torch.randn(10, requires_grad=False)
LG = SimpleLogisticRegression(1, w, b).to(device)

In [25]:
# Note: we also need to move data when asking for a prediction
LG(xb.to(device)).shape

torch.Size([8, 10])

Train and evaluate the network with forward gradient

In [26]:
def accuracy(net, loader, device):
  # A function that aggregates the accuracy over all mini-batches in the loader.
  # See here for a quick-start on torchmetrics: https://torchmetrics.readthedocs.io/en/stable/pages/quickstart.html.
  #acc = torchmetrics.Accuracy().to(device)
  acc = torchmetrics.Accuracy('multiclass', num_classes=10).to(device)
  for xb, yb in loader:
      xb, yb = xb.to(device)/255, yb.to(device)
      ypred = LG(xb)
      _ = acc(ypred, yb)
  return acc.compute()

In [27]:
# Average accuracy at initialization is 10% (random guessing).
accuracy(LG, test_loader, device)

tensor(0.0990, device='cuda:0')

Define Cross Entropy

In [28]:
# Note: it is important to move the CNN to the device before initializing the optimizer,
# since the optimizer also has a state that must be moved to the GPU.
loss = nn.CrossEntropyLoss()

In [29]:
def beale_function(x):
  return (torch.pow(torch.tensor([1.5])-x[0]+x[0]*x[1],2) + torch.pow(torch.tensor([2.25])-x[0]+x[0]*torch.pow(x[1],2),2)+torch.pow(torch.tensor([2.625])-x[0]+x[0]*torch.pow(x[1],3),2))

In [30]:
def rosenbrock_function(x):
  sum=0
  for i in range (x.size(0) -1):
    sum += (100*torch.pow(x[i+1] - torch.pow(x[i], 2), 2) + torch.pow(x[i]-1, 2))
  return sum

In [31]:
from functorch import jvp

In [32]:
def train_fwd_gradient(x, y):
  x, y = x.to(device), y.to(device)

  l_rate0 = 0.001
  f = rosenbrock_function

  #Parameters
  w = torch.FloatTensor(8, 28*28).uniform_(-1, 1)
  #w = torch.randint(-1, 1, (8, 28*28), dtype=torch.float32, requires_grad=False)
  #w = torch.div(w, torch.norm(w, 2))

  b = torch.FloatTensor(8).uniform_(-1, 1)
  #b = torch.randint(-1, 1, (8, ), dtype=torch.float32, requires_grad=False)
  #b = torch.div(b, torch.norm(b, 2))

  LG = SimpleLogisticRegression(1, w, b).to(device)
  w1 = w.reshape(-1)
  #w1 = torch.cat((w1, b), 0)

  error = torch.norm(LG(x) - y, 2)
  t=torch.tensor([0])

  while (error>1e-3) :

    t=t+1
    vw1=torch.diagonal(torch.normal(torch.zeros_like(w1),torch.eye(w1.shape[0])))
    vb=torch.diagonal(torch.normal(torch.zeros_like(b),torch.eye(b.shape[0])))

    ftw1 = f(w1)
    ftb = f(b)
    print('ftw1', ftw1)
    print('ftb', ftb)
    dtw1 = torch.tensor(jvp(f,(w1, ), (vw1, ))[1])
    dtb = torch.tensor(jvp(f,(b, ), (vb, ))[1])
    print('dtw1', dtw1)
    print('dtb', dtb)
    gtw1=vw1*dtw1
    gtb = vb*dtb
    w1 = w1 - l_rate0*gtw1
    b = b - l_rate0*gtb

    w = w.reshape(-1, 28*28)

    LG = SimpleLogisticRegression(1, w, b).to(device)
    error = torch.norm(LG(x) - y, 2)
    print('error', error)

  return w, b

In [33]:
for epoch in range(1):

  LG.train()
  for i in range(1):
    xb, yb = next(iter(train_loader))
    xb = xb.to(device)
    yb = yb.to(device)

    w, b = train_fwd_gradient(xb, yb)
    LG = SimpleLogisticRegression(1, w, b)

ftw1 tensor(338854.2500)
ftb tensor(219.6079)


<ipython-input-32-fea5583436d3>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dtw1 = torch.tensor(jvp(f,(w1, ), (vw1, ))[1])
<ipython-input-32-fea5583436d3>:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dtb = torch.tensor(jvp(f,(b, ), (vb, ))[1])


dtw1 tensor(-7605.6206)
dtb tensor(130.9954)
error tensor(33.4876, device='cuda:0', grad_fn=<NormBackward1>)
ftw1 tensor(6.4859e+09)
ftb tensor(269.6541)
dtw1 tensor(19696348.)
dtb tensor(254.6136)
error tensor(33.4837, device='cuda:0', grad_fn=<NormBackward1>)
ftw1 tensor(2.9799e+23)
ftb tensor(315.3481)
dtw1 tensor(-1.7735e+18)
dtb tensor(394.6307)
error tensor(33.4803, device='cuda:0', grad_fn=<NormBackward1>)
ftw1 tensor(inf)
ftb tensor(243.5589)
dtw1 tensor(nan)
dtb tensor(52.6054)
error tensor(33.4790, device='cuda:0', grad_fn=<NormBackward1>)
ftw1 tensor(nan)
ftb tensor(243.8828)
dtw1 tensor(nan)
dtb tensor(18.8531)
error tensor(33.4800, device='cuda:0', grad_fn=<NormBackward1>)
ftw1 tensor(nan)
ftb tensor(243.8987)
dtw1 tensor(nan)
dtb tensor(859.9277)
error tensor(33.3878, device='cuda:0', grad_fn=<NormBackward1>)
ftw1 tensor(nan)
ftb tensor(2315.8528)
dtw1 tensor(nan)
dtb tensor(-4173.4341)
error tensor(33.6830, device='cuda:0', grad_fn=<NormBackward1>)
ftw1 tensor(nan)
ftb t